# Example File:
In this package, we show three examples: 
<ol>
    <li>4 site XY model</li>
    <li><b>4 site Transverse Field XY model with random coefficients</b></li>
    <li> Custom Hamiltonian from OpenFermion </li>
</ol>

## Clone and Install The Repo via command line:
```
git clone https://github.com/kemperlab/cartan-quantum-synthesizer.git 
  
cd ./cartan-quantum-synthesizer/ 
  
pip install . 
```

## Import the Classes

In [54]:
from CQS.methods import *
#We will also use:
import numpy as np

# 4 Site TFXY Model

We first define our relevant variables, then create the three objects using default settings

# Example File:
In this package, we show three examples: 
<ol>
    <li>4 site XY model</li>
    <li><b>4 site Transverse Field XY model with random coefficients</b></li>
    <li> Custom Hamiltonian from OpenFermion </li>
</ol>

## Clone and Install The Repo via command line:
```
git clone https://github.com/kemperlab/cartan-quantum-synthesizer.git 
  
cd ./cartan-quantum-synthesizer/ 
  
pip install . 
```


## Import the Classes

In [55]:
from CQS.methods import *
#We will also use:
import numpy as np

# Transverse Field XY Model:
As an example of usage and customization, we go through the decompositon of the TFXY model used in the paper [Fixed Depth Hamiltonian Simulation via Cartan Decomposition](https://arxiv.org/abs/2104.00728). Note that in the paper only the transverse field is randomized, while in this example the entire Hamiltonian is randomized

In [56]:
#Define the system parameters
sites = 4
model = 'tfxy'
coefficient = 1
# This will be used to generate the Hamiltonian 1*(XXII + YYII + IXXI + IYYI + IIXX + IIYY + ZIII + IZII + IIZI + IIIZ)
modelTuple = [(coefficient, model)]
# In this case, we actually want random coefficients, except we also want to know the order of the Hamiltonian terms, and the number of random variables we need. So, we first generate temporary Hamiltonian to learn a out the terms that are generated

## Hamiltonian:

Now, we can create a Hamiltonian object. The generates the information about the Hamiltonian for the system to be simulated


In [57]:
tfxyH = Hamiltonian(sites, name=modelTuple)
# The Hamiltonian formats Pauli Strings using Tuples. For Example, XXII is represented by (1, 1, 0, 0) and IXYZX would be (0, 1, 2, 3, 1)

# We can count the number of terms as:
Hlen = len(tfxyH.HCoefs)

# And generate a list of random numbers via:
HRandCo = [np.random.rand(1)[0] for i in range(Hlen)]

# Then generate a new Hamiltonian objects as:
modelTuple = [(HRandCo, model)]
tfxyH = Hamiltonian(sites, name=modelTuple)


### Customizing the Hamiltonian

In [58]:
# You can add and remove terms using:
tfxyH.addTerms((1, (0,1,2,3)))
print('Includes IXYZ:')
tfxyH.getHamiltonian(type='printText')
tfxyH.removeTerm((0,1,2,3))
print('Removes IXYZ:')
tfxyH.getHamiltonian(type='printText')

#It is possible to add many terms at once using addTerms by passing a list of coefficients and Tuples like .addterms(([coefficients],[PauliStrings])). This may be useful if you generate the Hamiltonian using a different package or manually. However, the PauliStrings terms must first be converted to the Tuple format useable by this package: (PauliString)
#For Example:
HDict = {
    (1,1,0,0): 2,
    (2,1,2,0): 3
}
tfxyH.addTerms(([i for i in HDict.values()], [i for i in HDict.keys()]))
print("The dictionary has been added")
tfxyH.getHamiltonian(type='printText')
for key in HDict.keys():
    print(key)
    tfxyH.removeTerm(key)
print("The dictionary has been removed")
tfxyH.getHamiltonian(type='printText')
#Notice, the full XXII term was removed, not just the coefficient in the tuple. This is the expected behavior. If the user does not want to fully remove the term of the Hamiltonian, it is possible to pull the information about the tuple and coefficient using tfxyH.HCoefs.index() and then pulling out the coefficient and tuple pair to add it back later after the term is removed. 

#In this case we do care about that XXII term, so we'll add it back:
tfxyH.addTerms((np.random.rand(1)[0], (1,1,0,0)))

#Finally, we have:
tfxyH.getHamiltonian(type='printText')

Includes IXYZ:
0.5899855256470289 * ZIII
0.1077140219420698 * IZII
0.7345142800938713 * IIZI
0.15074360138194554 * IIIZ
0.9983805389781897 * XXII
0.06018904314139906 * YYII
0.4465036782091626 * IXXI
0.22741145709019084 * IYYI
0.8562077088113247 * IIXX
0.1047913513261679 * IIYY
1 * IXYZ
Removes IXYZ:
0.5899855256470289 * ZIII
0.1077140219420698 * IZII
0.7345142800938713 * IIZI
0.15074360138194554 * IIIZ
0.9983805389781897 * XXII
0.06018904314139906 * YYII
0.4465036782091626 * IXXI
0.22741145709019084 * IYYI
0.8562077088113247 * IIXX
0.1047913513261679 * IIYY
The dictionary has been added
0.5899855256470289 * ZIII
0.1077140219420698 * IZII
0.7345142800938713 * IIZI
0.15074360138194554 * IIIZ
2.9983805389781897 * XXII
0.06018904314139906 * YYII
0.4465036782091626 * IXXI
0.22741145709019084 * IYYI
0.8562077088113247 * IIXX
0.1047913513261679 * IIYY
3 * YXYI
(1, 1, 0, 0)
(2, 1, 2, 0)
The dictionary has been removed
0.5899855256470289 * ZIII
0.1077140219420698 * IZII
0.7345142800938713 * IIZ

## Cartan:

Pass the Hamiltonian object to the Cartan Object. It will perform a Cartan involution on the Hamiltonian Algebra generated by the Hamiltonain terms

In [59]:
# This defaults to the evenOdd Decomposition. In this case, it is not valid.

try:
    tfxyC = Cartan(tfxyH)
except Exception as e:
    print(e)
    
# We change the involution to one that partitions based on Y instead of I:
tfxyC = Cartan(tfxyH, involution='countY')
print("CountY Decomposition:")
print('Hamiltonian Algebra:')
print(tfxyC.g)
print('k:')
print(tfxyC.k)
print('h:')
print(tfxyC.h)



Invalid Involution. Please Choose an involution such that H ⊂ m
CountY Decomposition:
Hamiltonian Algebra:
[(1, 2, 0, 0), (2, 1, 0, 0), (0, 2, 1, 0), (0, 1, 2, 0), (0, 0, 2, 1), (0, 0, 1, 2), (2, 3, 1, 0), (0, 1, 3, 2), (0, 2, 3, 1), (1, 3, 2, 0), (1, 3, 3, 2), (2, 3, 3, 1), (3, 0, 0, 0), (0, 3, 0, 0), (0, 0, 3, 0), (0, 0, 0, 3), (2, 2, 0, 0), (0, 1, 1, 0), (0, 2, 2, 0), (0, 0, 1, 1), (0, 0, 2, 2), (1, 1, 0, 0), (1, 3, 1, 0), (2, 3, 2, 0), (0, 2, 3, 2), (0, 1, 3, 1), (2, 3, 3, 2), (1, 3, 3, 1)]
k:
[(1, 2, 0, 0), (2, 1, 0, 0), (0, 2, 1, 0), (0, 1, 2, 0), (0, 0, 2, 1), (0, 0, 1, 2), (2, 3, 1, 0), (0, 1, 3, 2), (0, 2, 3, 1), (1, 3, 2, 0), (1, 3, 3, 2), (2, 3, 3, 1)]
h:
[(3, 0, 0, 0), (0, 3, 0, 0), (0, 0, 3, 0), (0, 0, 0, 3)]


In [60]:
#To change the terms in the Cartan Subalgebra h, pass a list of commuting terms that exist in m. 
print('m terms:')
print(tfxyC.m)
tfxyC.subAlgebra(seedList=[(0,1,1,0),(1,1,0,0)])
print('New Ordered Hamiltonian Algebra:')
print(tfxyC.g)
print('New h:')
print(tfxyC.h)

#Reset back to default:
tfxyC.subAlgebra(seedList=[(3,0,0,0),(0,3,0,0)])


m terms:
[(3, 0, 0, 0), (0, 3, 0, 0), (0, 0, 3, 0), (0, 0, 0, 3), (2, 2, 0, 0), (0, 1, 1, 0), (0, 2, 2, 0), (0, 0, 1, 1), (0, 0, 2, 2), (1, 1, 0, 0), (1, 3, 1, 0), (2, 3, 2, 0), (0, 2, 3, 2), (0, 1, 3, 1), (2, 3, 3, 2), (1, 3, 3, 1)]
New Ordered Hamiltonian Algebra:
[(1, 2, 0, 0), (2, 1, 0, 0), (0, 2, 1, 0), (0, 1, 2, 0), (0, 0, 2, 1), (0, 0, 1, 2), (2, 3, 1, 0), (0, 1, 3, 2), (0, 2, 3, 1), (1, 3, 2, 0), (1, 3, 3, 2), (2, 3, 3, 1), (0, 1, 1, 0), (1, 1, 0, 0), (0, 0, 0, 3), (2, 3, 2, 0), (3, 0, 0, 0), (0, 3, 0, 0), (0, 0, 3, 0), (2, 2, 0, 0), (0, 2, 2, 0), (0, 0, 1, 1), (0, 0, 2, 2), (1, 3, 1, 0), (0, 2, 3, 2), (0, 1, 3, 1), (2, 3, 3, 2), (1, 3, 3, 1)]
New h:
[(0, 1, 1, 0), (1, 1, 0, 0), (0, 0, 0, 3), (2, 3, 2, 0)]


## Implementing Appendix E simplifications:
In Appendix E, we introduce a simplification of the k algebra we call piling. To implement this procedure using the CQS package, we first define a custom function then redefine the k algebra in the Cartan object, like so:

In [61]:
def pile(N):
    k = []
    for i in range(N-1):
        for j in range(N-i-1):
            elem = (0,)*j+(2,1)+(0,)*(N-j-2)
            k.append(elem)
            
            elem = (0,)*j+(1,2)+(0,)*(N-j-2)
            k.append(elem)
    return k
tfxyC.k = pile(tfxyH.sites)
print('New piled K. This is an equivalent, simplified algebra.')
print(tfxyC.k)

New piled K. This is an equivalent, simplified algebra.
[(2, 1, 0, 0), (1, 2, 0, 0), (0, 2, 1, 0), (0, 1, 2, 0), (0, 0, 2, 1), (0, 0, 1, 2), (2, 1, 0, 0), (1, 2, 0, 0), (0, 2, 1, 0), (0, 1, 2, 0), (2, 1, 0, 0), (1, 2, 0, 0)]


### Find Parameters:

Finding the parameters is the expensive part of the package. The default method is to use gradient decent via BFGS optimization in the `scipy.otpimize` package. 

In [62]:
#Generate the Parameters via:
tfxyP = FindParameters(tfxyC)

#printResult() returns the parameters, the error produced by removing invalid terms, and the normed difference of the Cartan and the exact matrix exponentiation. 
tfxyP.printResult()

         Current function value: -1.148064
         Iterations: 4
         Function evaluations: 65
         Gradient evaluations: 53
--- 2.004863739013672 seconds ---
Optimization Error:
1.071540772737448
Printing Results:
K elements 

0.46014648247471296 *YXII
-0.048791696563937836*XYII
-0.0966106265988898 *IYXI
-0.03284945700873765*IXYI
0.7183839679698757  *IIYX
0.6187408487011098  *IIXY
0.6018820076391309  *YXII
-0.03156881578208815*XYII
-0.18759754903425124*IYXI
0.07256002508725892 *IXYI
0.512910061317558   *YXII
-0.05179935530028888*XYII

 h elements: 
 
(-0.49667737023075614+0j)*ZIII
(-0.23503346963451557+0j)*IZII
(-0.34822820685015526+0j)*IIZI
(-0.7415404760349719+0j) *IIIZ
Normed Error |KHK - Exact|:
3.6442453020568135


In [63]:
#The results of the optimization are stored as
print(tfxyP.cartan.h)
print(tfxyP.hCoefs)

print(tfxyP.cartan.k)
print(tfxyP.kCoefs)


[(3, 0, 0, 0), (0, 3, 0, 0), (0, 0, 3, 0), (0, 0, 0, 3)]
[(-0.49667737023075614+0j), (-0.23503346963451557+0j), (-0.34822820685015526+0j), (-0.7415404760349719+0j)]
[(2, 1, 0, 0), (1, 2, 0, 0), (0, 2, 1, 0), (0, 1, 2, 0), (0, 0, 2, 1), (0, 0, 1, 2), (2, 1, 0, 0), (1, 2, 0, 0), (0, 2, 1, 0), (0, 1, 2, 0), (2, 1, 0, 0), (1, 2, 0, 0)]
[ 0.46014648 -0.0487917  -0.09661063 -0.03284946  0.71838397  0.61874085
  0.60188201 -0.03156882 -0.18759755  0.07256003  0.51291006 -0.05179936]
